In [1]:
library('DESeq2')
library('viridis')
library('ggplot2')
library('ggpubr')

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min


Attaching package: ‘S4Vectors’

The followin

# Assemble Metadata

In [81]:
#### define palettes ####

levels(treatments$ROOTSTOCK)

[1] "1103P"       "1103PAULSEN" "3309C"       "OWN"         "SO4"

In [134]:
#### Prepare Metadata required data ####

treatments <- read.csv('./1719_geneExpression_metadata.csv')
treatments$Year <- factor(treatments$COLLECTED_DATE, 
                          levels=c('5/22/17', '7/30/17', '9/25/17', '5/30/18', '1/8/18', '9/18/18', '5/23/19', '9/5/19', '9/17/19'), 
                          labels=c('2017','2017','2017', '2018','2018','2018', '2019','2019','2019'))
treatments$Tissue <- factor(treatments$TISSUE, 
                            levels=c('FLOWER', 'GREEN_BERRY', 'LEAF', 'RIPE_BERRY'),
                            labels=c('Reproductive', 'Reproductive', 'Leaf', 'Reproductive'))
treatments$Phenology <- factor(treatments$COLLECTED_DATE, 
                               levels=c('5/22/17', '7/30/17', '9/25/17', '5/30/18', '1/8/18', '9/18/18', '5/23/19', '9/5/19', '9/17/19'), 
                               labels=c('Anthesis','Veraison','Harvest', 'Anthesis','Veraison','Harvest', 'Anthesis','Veraison','Harvest'))
treatments$indexer <- paste(treatments$Year, treatments$Phenology, treatments$Tissue, treatments$ROW, treatments$BLOCK, treatments$VINE, sep='_')
treatments$Rootstock <- factor(treatments$ROOTSTOCK, levels=c('OWN', '1103PAULSEN', '1103P', '3309C', 'SO4'), labels=c('Ungrafted', '1103P', '1103P', '3309C', 'SO4'))
treatments$Irrigation <- factor(treatments$TREATMENT, levels=c('NONE', 'RDI', 'FULL'), labels=c('None', 'RDI', 'Full'))
treatments$Row <- treatments$ROW

#Try to make the sample name in this file look like mine without python...
# warning suggest NAs introduced, but  don't see any...
treatments <- transform(treatments, sample_name=reshape::colsplit(SEQUENCE_NAME, split = "_", names = c('Y', 'N', 'T')))
treatments$sample_name.N <- as.numeric(as.character(treatments$sample_name.N))
treatments$sampleName <- paste(treatments$sample_name.Y, formatC(treatments$sample_name.N, width=3, flag="0"), treatments$sample_name.T, sep="_")

treatments <- treatments %>% dplyr::select(sampleName, Year:Row)
colnames(treatments) <- c('sampleName', 'Year', 'Tissue', 'Phenology', 'indexer', 'Rootstock', 'Irrigation', 'Row')
rownames(treatments) <- treatments$sampleName

treatments


Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

,sampleName,Year,Tissue,Phenology,indexer,Rootstock,Irrigation,Row
,<chr>,<fct>,<fct>,<fct>,<chr>,<fct>,<fct>,<int>
A1Y1_001_L,A1Y1_001_L,2017,Leaf,Anthesis,2017_Anthesis_Leaf_8_A_2,1103P,None,8
A1Y1_002_L,A1Y1_002_L,2017,Leaf,Anthesis,2017_Anthesis_Leaf_8_A_3,1103P,None,8
A1Y1_003_L,A1Y1_003_L,2017,Leaf,Anthesis,2017_Anthesis_Leaf_8_B_2,3309C,None,8
A1Y1_004_L,A1Y1_004_L,2017,Leaf,Anthesis,2017_Anthesis_Leaf_8_B_2,3309C,None,8
A1Y1_005_L,A1Y1_005_L,2017,Leaf,Anthesis,2017_Anthesis_Leaf_8_C_2,SO4,None,8
A1Y1_006_L,A1Y1_006_L,2017,Leaf,Anthesis,2017_Anthesis_Leaf_8_C_3,SO4,None,8
A1Y1_007_L,A1Y1_007_L,2017,Leaf,Anthesis,2017_Anthesis_Leaf_8_D_2,Ungrafted,None,8
A1Y1_008_L,A1Y1_008_L,2017,Leaf,Anthesis,2017_Anthesis_Leaf_8_D_3,Ungrafted,None,8
A1Y1_009_L,A1Y1_009_L,2017,Leaf,Anthesis,2017_Anthesis_Leaf_9_A_2,Ungrafted,RDI,9


In [135]:
treatments$sampleName[grepl("10_B_2", treatments$indexer)]

[1] "A1Y1_019_L"    "A1Y1_091_L"    "A1Y1_163_L"    "A1Y1_019_R"   
 [5] "A1Y1_091_R"    "A1Y1_163_R"    "A1Y2_235_L"    "A1Y2_307_L"   
 [9] "A1Y2_235_R"    "#N/A_ NA_#N/A" "A1Y2_379_L"    "A1Y3_451_L"   
[13] "A1Y3_451_R"    "A1Y3_595_L"

### Remove SO4 Overgrowth + Low Sequence Samples

In [137]:
#SO4 overgrowth
treatments <- treatments[treatments$sampleName != 'A1Y1_019_R',] 
treatments <- treatments[treatments$sampleName != 'A1Y1_019_L',] 
treatments <- treatments[treatments$sampleName != 'A1Y1_091_L',] 
treatments <- treatments[treatments$sampleName != 'A1Y1_091_R',] 
treatments <- treatments[treatments$sampleName != 'A1Y1_163_R',] 
treatments <- treatments[treatments$sampleName != 'A1Y1_163_L',]
treatments <- treatments[treatments$sampleName != 'A1Y2_235_R',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_235_L',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_307_R',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_307_L',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_379_R',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_379_L',] 
treatments <- treatments[treatments$sampleName != 'A1Y3_451_R',] #
treatments <- treatments[treatments$sampleName != 'A1Y3_451_L',] #
treatments <- treatments[treatments$sampleName != 'A1Y3_595_R',] #
treatments <- treatments[treatments$sampleName != 'A1Y3_595_L',] #
treatments <- treatments[treatments$sampleName != '#N/A_ NA_#N/A',] # deep existential sigh

#Remove samples that had less than 500K reads 
treatments <- treatments[treatments$sampleName != 'A1Y1_030_L',]
treatments <- treatments[treatments$sampleName != 'A1Y2_223_L',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_225_L',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_225_R',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_250_L',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_252_L',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_253_L',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_257_L',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_270_L',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_273_L',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_276_L',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_284_R',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_298_L',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_312_L',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_329_L',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_332_L',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_338_L',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_339_L',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_344_L',] #
treatments <- treatments[treatments$sampleName != 'A1Y2_358_L',] #

treatments$Row <- as.factor(treatments$Row)
treatments$Block <- rep('A', dim(treatments)[1])
treatments$Block[treatments$Row == 11] <- "B" 
treatments$Block[treatments$Row == 12] <- "B"
treatments$Block[treatments$Row == 13] <- "B"
treatments$Block[treatments$Row == 14] <- "C"
treatments$Block[treatments$Row == 15] <- "C"
treatments$Block[treatments$Row == 16] <- "C"
treatments$Block <- as.factor(treatments$Block)

# a couple aren't labelled with the right year in the sampleName...
treatments$sampleName[treatments$sampleName == 'A1Y1_217_L'] <- 'A1Y2_217_L'
treatments$sampleName[treatments$sampleName == 'A1Y1_218_L'] <- 'A1Y2_218_L'
treatments$sampleName[treatments$sampleName == 'A1Y1_219_L'] <- 'A1Y2_219_L'
treatments$sampleName[treatments$sampleName == 'A1Y1_220_L'] <- 'A1Y2_220_L'
treatments$sampleName[treatments$sampleName == 'A1Y1_221_L'] <- 'A1Y2_221_L'
treatments$sampleName[treatments$sampleName == 'A1Y1_222_L'] <- 'A1Y2_222_L'

rownames(treatments) <- treatments$sampleName

treatments

,sampleName,Year,Tissue,Phenology,indexer,Rootstock,Irrigation,Row,Block
,<chr>,<fct>,<fct>,<fct>,<chr>,<fct>,<fct>,<fct>,<fct>
A1Y1_001_L,A1Y1_001_L,2017,Leaf,Anthesis,2017_Anthesis_Leaf_8_A_2,1103P,None,8,A
A1Y1_002_L,A1Y1_002_L,2017,Leaf,Anthesis,2017_Anthesis_Leaf_8_A_3,1103P,None,8,A
A1Y1_003_L,A1Y1_003_L,2017,Leaf,Anthesis,2017_Anthesis_Leaf_8_B_2,3309C,None,8,A
A1Y1_004_L,A1Y1_004_L,2017,Leaf,Anthesis,2017_Anthesis_Leaf_8_B_2,3309C,None,8,A
A1Y1_005_L,A1Y1_005_L,2017,Leaf,Anthesis,2017_Anthesis_Leaf_8_C_2,SO4,None,8,A
A1Y1_006_L,A1Y1_006_L,2017,Leaf,Anthesis,2017_Anthesis_Leaf_8_C_3,SO4,None,8,A
A1Y1_007_L,A1Y1_007_L,2017,Leaf,Anthesis,2017_Anthesis_Leaf_8_D_2,Ungrafted,None,8,A
A1Y1_008_L,A1Y1_008_L,2017,Leaf,Anthesis,2017_Anthesis_Leaf_8_D_3,Ungrafted,None,8,A
A1Y1_009_L,A1Y1_009_L,2017,Leaf,Anthesis,2017_Anthesis_Leaf_9_A_2,Ungrafted,RDI,9,A


In [138]:
## sanity checks
dim(treatments)
table(treatments$Year)
table(treatments$Tissue)
table(treatments$Phenology)
table(treatments$Rootstock)
table(treatments$Irrigation)
table(treatments$Row)
table(treatments$Block)
write.csv(treatments, file='1719_geneExpressed_metadata_processedR.csv')

[1] 1179    9


2017 2018 2019 
 425  407  347 


Reproductive         Leaf 
         558          621 


Anthesis Veraison  Harvest 
     414      410      355 


Ungrafted     1103P     3309C       SO4 
      296       303       301       279 


None  RDI Full 
 401  397  381 


  8   9  10  11  12  13  14  15  16 
135 133 118 135 132 133 129 133 131 


  A   B   C 
386 400 393 

#  Assemble featureCount outputs into a matrix

In [112]:
# fun to process each file
readCountMatrix <- function(sample, dir){
    file <- paste(dir, sample, "_fc.counts", sep='')
    d <- read.table(file, sep='\t', header=1)
    colnames(d) <- c('gene', 'chr', 'start', 'end', 'strand', 'length', sample)
    rownames(d) <- d$gene
    d <- d %>% dplyr::select(sample)
    return(d)
}


#### create 2017 counts matrix
counts2017 <- list.files(path="2017_featureCounts/", pattern="*.counts$", recursive=FALSE)
counts2017 <- stringr::str_remove(counts2017, "_fc.counts")
x <- lapply(X=counts2017, FUN=readCountMatrix, dir="2017_featureCounts/")

counts_2017 <- x[[1]]
for (ix in 2:length(x)){
    counts_2017 <- merge(counts_2017, x[[ix]], by=0)
    rownames(counts_2017) <- counts_2017$Row.names
    counts_2017$Row.names <- NULL
}
dim(counts_2017)

#### create 2018 counts matrix
counts2018 <- list.files(path="2018_featureCounts/", pattern="*.counts$", recursive=FALSE)
counts2018 <- stringr::str_remove(counts2018, "_fc.counts")
x <- lapply(X=counts2018, FUN=readCountMatrix, dir="2018_featureCounts/")

counts_2018 <- x[[1]]
for (ix in 2:length(x)){
    counts_2018 <- merge(counts_2018, x[[ix]], by=0)
    rownames(counts_2018) <- counts_2018$Row.names
    counts_2018$Row.names <- NULL
}
dim(counts_2018)

#### create 2019 counts matrix
counts2019 <- list.files(path="2019_featureCounts/", pattern="A1Y3.*counts$", recursive=FALSE)
counts2019 <- stringr::str_remove(counts2019, "_fc.counts")
x <- lapply(X=counts2019, FUN=readCountMatrix, dir="2019_featureCounts/")

counts_2019 <- x[[1]]
for (ix in 2:length(x)){
    counts_2019 <- merge(counts_2019, x[[ix]], by=0)
    rownames(counts_2019) <- counts_2019$Row.names
    counts_2019$Row.names <- NULL
}

dim(counts_2019)

[1] 42413   432

[1] 42413   430

[1] 42413   349

In [139]:
counts <- merge(counts_2017, counts_2018, by=0)
rownames(counts) <- counts$Row.names
counts$Row.names <- NULL

counts <- merge(counts, counts_2019, by=0)
rownames(counts) <- counts$Row.names
counts$Row.names <- NULL

dim(counts)

[1] 42413  1211

## Filter samples not in metdata

In [140]:
counts <- counts[,(colnames(counts) %in% rownames(treatments))]
dim(counts)

[1] 42413  1178

In [141]:
treatments <- treatments[rownames(treatments) %in% colnames(counts),]
dim(treatments)

[1] 1178    9

In [142]:
#reorder for DESeq2 ?!
counts <- counts %>% dplyr::select(rownames(treatments))
dim(counts)

[1] 42413  1178

# Fit and Save DESeq2 _dds_ objects

In [144]:
#model cannot include row because row and irrigation correlate
dds <- DESeqDataSetFromMatrix(countData=as.matrix(counts), colData=treatments, 
                              design= ~Block + Irrigation + Year + Phenology + Rootstock + Tissue)
dds <- DESeq(dds)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


In [145]:
save(dds, file='/xfs2/zachary.n.harris/MtVernon_sam/2019/1719_full_model_dds.Rdata')

In [3]:
#on rerunning
#load('1718_full_model_dds.Rdata')

## Quick and dirty counting for various filtering

In [146]:
mat <- matrix(nrow=20, ncol=20)

for (count in 1:20){
    for (samples in 1:20){
        idx <- rowSums( counts(dds, normalized=TRUE) >= count ) >= samples
        genes <- rownames(dds)[idx]
        #remove gene always has weirdly high count
        idx <- idx[-which(genes == 'Vitvi19g01871')]
        genes <- genes[-which(genes == 'Vitvi19g01871')]
        mat[samples, count] <- length(genes)    
    }      
}

mat

37124,31876,28938,27180,26048,25229,24626,24173,23727,23395,22887,22640,22368,22131,21922,21717,21524,21362,21197,21035
34909,29685,27271,25738,24817,24133,23579,23191,22756,22440,22084,21827,21591,21354,21175,20950,20754,20586,20408,20240
33580,28702,26452,25129,24293,23644,23099,22680,22316,22017,21672,21409,21209,20977,20787,20579,20354,20166,19985,19809
32721,28090,25996,24769,23967,23324,22813,22419,22060,21762,21430,21213,20975,20730,20518,20305,20100,19907,19705,19523
32062,27631,25676,24508,23710,23116,22605,22253,21862,21576,21248,21009,20773,20534,20310,20102,19890,19696,19510,19342
31552,27315,25443,24302,23518,22933,22442,22084,21704,21415,21102,20855,20620,20349,20150,19945,19720,19534,19353,19185
31116,27026,25217,24133,23358,22792,22316,21946,21578,21279,20979,20726,20487,20224,20035,19831,19598,19407,19228,19063
30746,26767,25040,23963,23213,22662,22199,21822,21481,21146,20871,20613,20364,20129,19916,19687,19473,19291,19112,18955
30413,26566,24888,23839,23098,22556,22087,21719,21395,21047,20777,20506,20262,20036,19791,19581,19383,19189,19002,18866
30119,26398,24746,23741,23009,22474,21995,21630,21294,20969,20684,20413,20188,19940,19706,19493,19283,19099,18933,18787
29868,26245,24625,23634,22906,22389,21909,21553,21224,20882,20605,20333,20097,19843,19605,19396,19210,19018,18863,18704


In [147]:
mat <- matrix(nrow=10, ncol=10)

for (count in 31:40){
    for (samples in 31:40){
        idx <- rowSums( counts(dds, normalized=TRUE) >= count ) >= samples
        genes <- rownames(dds)[idx]
        idx <- idx[-which(genes == 'Vitvi19g01871')]
        genes <- genes[-which(genes == 'Vitvi19g01871')]
        mat[samples-30, count-30] <- length(genes)    
    }      
}

mat

16282,16169,16039,15938,15807,15701,15575,15471,15380,15280
16254,16143,16012,15904,15777,15667,15541,15441,15351,15241
16223,16114,15989,15874,15749,15628,15507,15408,15316,15210
16203,16073,15957,15848,15715,15593,15478,15380,15276,15165
16180,16044,15926,15808,15691,15570,15451,15351,15250,15137
16143,16016,15897,15779,15653,15541,15420,15326,15215,15115
16111,15973,15863,15739,15618,15511,15388,15296,15184,15089
16073,15957,15830,15706,15596,15478,15366,15267,15156,15065
16040,15929,15799,15687,15569,15453,15340,15243,15127,15041
16017,15900,15768,15660,15540,15430,15322,15215,15109,15012


## Actually filter and refit

In [148]:
#require count = 4 in > 4 samples
idx <- rowSums( counts(dds, normalized=TRUE) >= 4 ) >= 4
genes <- rownames(dds)[idx]

#remove persistent outlier gene
idx <- idx[-which(genes == 'Vitvi19g01871')]
genes <- genes[-which(genes == 'Vitvi19g01871')]
length(genes)

[1] 24769

In [150]:
#re-estimate
dds_filt <- dds[idx,]
dds_filt <- DESeq(dds_filt)

using pre-existing size factors
estimating dispersions
found already estimated dispersions, replacing these
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


In [151]:
# write
save(dds_filt, file='/xfs2/zachary.n.harris/MtVernon_sam/2019/1719_filt_model_dds.Rdata')

# Transform for downstream analysis

In [153]:
#### Data Viz ####

vsd <- vst(dds_filt, blind=FALSE)
vsd <- assay(vsd)

#vsd_counts <- assay(vsd)
vsd_counts <- as.data.frame(t(vsd))
dim(vsd_counts)

vsd_counts_varFilt <- vsd_counts[,apply(vsd_counts, 2, sd) > 0]
dim(vsd_counts_varFilt)

save(vsd_counts_varFilt, file='/xfs2/zachary.n.harris/MtVernon_sam/2019/1719_VSD_counts_varFilt.Rdata')

[1]  1178 24770

[1]  1178 24392

In [154]:
write.csv(treatments, file='1719_treatments.csv')